# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
# Create Dataframe from weatherpy cities csv 
weather_cities_df=pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_cities_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,San Cristobal,VE,7.7669,-72.2250,77.18,57,100,3.00
1,Phan Thiet,VN,10.9333,108.1000,75.97,89,100,3.98
2,Isangel,VU,-19.5500,169.2667,76.62,77,100,13.09
3,Norman Wells,CA,65.2820,-126.8329,19.44,93,75,2.30
4,Atuona,PF,-9.8000,-139.0333,77.18,70,3,18.77
...,...,...,...,...,...,...,...,...
567,Te Anau,NZ,-45.4167,167.7167,52.66,80,100,5.39
568,Nueve de Julio,AR,-35.4444,-60.8831,78.53,45,99,12.39
569,Gijang,KR,35.2442,129.2139,46.13,59,67,7.74
570,Gorontalo,ID,0.5412,123.0595,76.01,91,99,2.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
humidity = weather_cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_cities_df[["Latitude", "Longitude"]]

In [20]:
#Plot Map
fig = gmaps.figure()

#Add Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)

#Show Map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Create new dataframe with sorted criteria 
weather_df=weather_cities_df.loc[(weather_cities_df["Max Temperature"] > 70) & (weather_cities_df["Max Temperature"] < 80) \
                                 & (weather_cities_df["Cloudiness"] == 0) & (weather_cities_df["Wind Speed"]<10), :]
weather_df=weather_df.dropna(how='any')
weather_df.reset_index(inplace=True)

# delete index column 
del weather_df["index"]

weather_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Oberá,AR,-27.4871,-55.1199,78.85,48,0,2.75
1,Terrasini,IT,38.1460,13.0844,74.08,82,0,3.44
2,Dwārka,IN,22.2394,68.9678,79.65,61,0,9.26
3,Rohri,PK,27.6922,68.8951,71.47,78,0,0.00
4,Jamestown,US,42.0970,-79.2353,71.24,44,0,4.61
5,San Patricio,US,28.0170,-97.5169,78.75,20,0,9.82
6,Saint-Pierre,RE,-21.3393,55.4781,71.91,73,0,4.61
7,Kumluca,TR,36.3703,30.2869,71.04,47,0,4.92
8,Marmari,GR,38.0484,24.3204,71.74,47,0,1.01
9,Tupanciretã,BR,-29.0806,-53.8358,73.51,63,0,6.69


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
#Create List
hotel_df = []

#Loop to find hotels for each city
for i in range(len(weather_df)):
    lat = weather_df.loc[i]['Latitude']
    lng = weather_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
        
#Add Hotel Name column       
weather_df["Hotel Name"] = hotel_df


weather_df = weather_df.dropna(how='any')

weather_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Oberá,AR,-27.4871,-55.1199,78.85,48,0,2.75,Oberá
1,Terrasini,IT,38.1460,13.0844,74.08,82,0,3.44,Terrasini
2,Dwārka,IN,22.2394,68.9678,79.65,61,0,9.26,Dwarka
3,Rohri,PK,27.6922,68.8951,71.47,78,0,0.00,Sukkur
4,Jamestown,US,42.0970,-79.2353,71.24,44,0,4.61,Jamestown
5,San Patricio,US,28.0170,-97.5169,78.75,20,0,9.82,Sinton
6,Saint-Pierre,RE,-21.3393,55.4781,71.91,73,0,4.61,Saint Pierre
7,Kumluca,TR,36.3703,30.2869,71.04,47,0,4.92,Kumluca
8,Marmari,GR,38.0484,24.3204,71.74,47,0,1.01,Marmari
9,Tupanciretã,BR,-29.0806,-53.8358,73.51,63,0,6.69,Tupanciretã


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in weather_df.iterrows()]
locations = weather_df[["Latitude", "Longitude"]]

In [29]:
# Add marker layer on top of heat map
markers=gmaps.marker_layer(locations)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))